# Module 5: Assignment 2
------------------------

Run-length encoding (RLE) is a form of lossless data compression in which runs of data (sequences in which the same data value occurs in many consecutive data elements) are stored as a single data value and count, rather than as the original run. For example,

```
Original: AAAAAAAAAAAAABBBCCD (19 characters), 
Encoded:  13A3B2C1D (7 characters)
```


This is most efficient on data that contains many such runs, for example, simple graphic images such as icons, line drawings, Conway's Game of Life, and animations. For files that do not have many runs, RLE could increase the file size.

```
Original: ABCABCABC(9 characters), 
Encoded:  A1B1C1A1B1C1A1B1C1 (18 characters)
```


RLE may also be used to refer to an early graphics file format supported by CompuServe for compressing black and white images, but was widely supplanted by their later Graphics Interchange Format (GIF). Check out this great throwback article, [Smile You're on RLE](http://csbruce.com/cbm/transactor/pdfs/trans_v7_i06.pdf).

### Lossless Compression in DNA Sequences

Lossless compression algorithms have been used for efficiently storing DNA seqeunces, but the tried-and-true `.fasta` text file has never been usurped as the standard. Take a look at some of the following approaches:
* [A Compression Algorithm for DNA Sequences and Its Applications in Genome Comparison - PubMed](https://pubmed.ncbi.nlm.nih.gov/11072342/)
* [A lossless compression algorithm for DNA sequences - PubMed](https://pubmed.ncbi.nlm.nih.gov/19887334/)
* [Biological sequence compression algorithms - PubMed](https://pubmed.ncbi.nlm.nih.gov/11700586/)



Develop a lossless compression algorithm based on RLE for storing DNA sequences. You may choose to implement an existing algorithm, modify one, or create your own. Note that there are some properties of DNA, such as long repeats, that can be exploited to gain greater compression over single letter runs.

The only criteria is that your approach must be able to fully decode as the original sequence and header AND it must never be larger size than the original. Your program should be able to reade/write the encoded and decoded sequence to a file.

Test your approach using Human Chromosome 21 sequence and the Sars-CoV-2 genome. 


Calculate the compression ratio of your approach for each sequence.

$ Compression\ ratio  = (Size_{before})/(Size_{after})$

### RLE Approach

In [11]:
def encode_rle(sequence):
    encoded = []
    i = 0
    while i < len(sequence):
        count = 1
        while i + 1 < len(sequence) and sequence[i] == sequence[i + 1]:
            count += 1
            i += 1
        if count > 1:
            encoded.append(f"{count}{sequence[i]}")
        else:
            encoded.append(sequence[i])
        i += 1
    enc = ''.join(encoded)
    return enc


def decode_rle(encoded_sequence):
    decoded = []
    i = 0
    while i < len(encoded_sequence):
        # Check if the current character is a digit, indicating the start of a count
        if encoded_sequence[i].isdigit():
            count_str = ''
            # Collect all consecutive digits to handle multi-digit counts
            while i < len(encoded_sequence) and encoded_sequence[i].isdigit():
                count_str += encoded_sequence[i]
                i += 1
            count = int(count_str)
            # Append the next character count times
            decoded.append(encoded_sequence[i] * count)
        else:
            # Handle single characters without a preceding count
            decoded.append(encoded_sequence[i])
        i += 1
    dec = ''.join(decoded)
    return dec


### File Manipulation and Computations

In [8]:

def load_fasta(file_path):
    '''Load a DNA sequence from a FASTA file'''
    header = ""
    sequence = []
    
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                # Header line
                header = line
            else:
                # Sequence lines
                sequence.append(line)
    
    # Join sequence lines into a single string
    sequence = ''.join(sequence)
    return header, sequence


def calculate_compression_ratio(original, encoded):
    comp_ratio = len(original) / len(encoded)
    return comp_ratio
    
def compress_fasta(input_file, compressed_file):
    # Load the FASTA file
    header, sequence = load_fasta(input_file)
    
    # Compress the sequence
    encoded_sequence = encode_rle(sequence)
    
    # Save the compressed sequence to a new file with the header
    with open(compressed_file, 'w') as file:
        file.write(header + '\n')
        file.write(encoded_sequence + '\n')
    
    # Calculate compression ratio
    ratio = calculate_compression_ratio(sequence, encoded_sequence)
    print(f"Compression Ratio: {ratio}")
    
    return encoded_sequence


def decompress_fasta(compressed_file, decompressed_file):
    # Load the compressed FASTA file
    with open(compressed_file, 'r') as file:
        header = file.readline().strip()
        encoded_sequence = file.readline().strip()
    
    # Decode the sequence
    decoded_sequence = decode_rle(encoded_sequence)
    
    # Save the decompressed sequence to a new FASTA file with the header
    with open(decompressed_file, 'w') as file:
        file.write(header + '\n')
        file.write(decoded_sequence + '\n')
    
    return decoded_sequence




Compression Ratio: 1.0746696035242291
CCCATGTGATTTTAATAGCTTCTCAGGAGAATGACAAAAAAAAAAAAAAA
CCCATGTGATTTTAATAGCTTCTCAGGAGAATGACAAAAAAAAAAAAAAA
Compression and decompression verified successfully.


### Chromosome 21

In [9]:

# Example usage
input_fasta_file = "./data/chromosome-21/chr21-genes.fasta"  # Path to the original chr21 FASTA file
compressed_fasta_file = "./data/chromosome-21/compressed_chr21.fasta"  # File to save the compressed sequence
decompressed_fasta_file = "./data/chromosome-21/decomp_chr21.fasta"  # File to save the decompressed sequence

# Compress the sequence from the FASTA file
encoded_sequence = compress_fasta(input_fasta_file, compressed_fasta_file)

# Decompress the sequence for verification
decoded_sequence = decompress_fasta(compressed_fasta_file, decompressed_fasta_file)

# Verify that decompression returns the original sequence
_, original_sequence = load_fasta(input_fasta_file)

print(original_sequence[-50:])
print(decoded_sequence[-50:])
assert original_sequence == decoded_sequence, "Decompressed sequence does not match the original sequence!"
print("Compression and decompression verified successfully.")


Compression Ratio: 1.078411399385881
CCCCTCAGCCCTCAGGCCTTCATCTCTCCTGGCCCATCTTCCTACTCTGA
CCCCTCAGCCCTCAGGCCTTCATCTCTCCTGGCCCATCTTCCTACTCTGA
Compression and decompression verified successfully.


### Sars CoV 2

In [10]:

input_fasta_file = "./data/sars-cov-2/human_sars-cov.fasta"  # Path to the original Sars CoV 2 FASTA file
compressed_fasta_file = "./data/sars-cov-2/compressed_sars.fasta"  # File to save the compressed sequence
decompressed_fasta_file = "./data/sars-cov-2/decomp_sars.fasta"  # File to save the decompressed sequence

# Compress the sequence from the FASTA file
encoded_sequence = compress_fasta(input_fasta_file, compressed_fasta_file)

# Decompress the sequence for verification
decoded_sequence = decompress_fasta(compressed_fasta_file, decompressed_fasta_file)

# Verify that decompression returns the original sequence
_, original_sequence = load_fasta(input_fasta_file)

print(original_sequence[-50:])
print(decoded_sequence[-50:])
assert original_sequence == decoded_sequence, "Decompressed sequence does not match the original sequence!"
print("Compression and decompression verified successfully.")


Compression Ratio: 1.0746696035242291
CCCATGTGATTTTAATAGCTTCTCAGGAGAATGACAAAAAAAAAAAAAAA
CCCATGTGATTTTAATAGCTTCTCAGGAGAATGACAAAAAAAAAAAAAAA
Compression and decompression verified successfully.


### Results
Compression rates are greater than 1, so this is good!